# Checking and describing the generated data

It is always beneficial to add a notebook that quickly looks into the data to help you remember, which data you collected and if it actually looks correct.

In [1]:
from algbench import describe, read_as_pandas, Benchmark
from _conf import EXPERIMENT_DATA

In [2]:
describe(EXPERIMENT_DATA)

An entry in the database can look like this:
_____________________________________________
 result:
| num_nodes: 264
| lower_bound: 49134.999999999985
| objective: 49134.99999999999
 timestamp: 2023-11-18T13:03:38.267844
 runtime: 7.737646818161011
 stdout: [[0.018755435943603516, 'Set parameter Username\n'], [0.020169496536254883, '...
 stderr: []
 logging: [{'name': 'Evaluation', 'msg': 'Building model.', 'args': [], 'levelname': 'I...
 env_fingerprint: dc522e14e66ed51d57852d501a70f277a707ab31
 args_fingerprint: e0081c8b67d95fc78986cc58ecae061c7c08b64a
 parameters:
| func: run_solver
| args:
|| instance_name: pr264
|| time_limit: 90
|| strategy: GurobiTspSolver
|| opt_tol: 0.001
 argv: ['/ibr/home/krupke/anaconda3/envs/mo310/lib/python3.10/site-packages/slurmina...
 env:
| hostname: algra02
| python_version: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
| python: /ibr/home/krupke/anaconda3/envs/mo310/bin/python3
| cwd: /misc/ibr/home/krupke/cpsat-primer/examples/tsp_evaluation_t

In [3]:
t = read_as_pandas(
    EXPERIMENT_DATA,
    lambda entry: {
        "instance_name": entry["parameters"]["args"]["instance_name"],
        "num_nodes": entry["result"]["num_nodes"],
        "time_limit": entry["parameters"]["args"]["time_limit"],
        "strategy": entry["parameters"]["args"]["strategy"],
        "opt_tol": entry["parameters"]["args"]["opt_tol"],
        "runtime": entry["runtime"],
        "objective": entry["result"]["objective"],
        "lower_bound": entry["result"]["lower_bound"],
    },
)
t.drop_duplicates(
    inplace=True, subset=["instance_name", "num_nodes", "strategy", "opt_tol"]
)
t["opt_gap"] = (t["objective"] - t["lower_bound"]) / t["lower_bound"]
t.sort_values(["num_nodes", "instance_name"])

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
300,att48,48,90,GurobiTspSolver,0.001,0.094591,33522.0,33522.0,0.000000
301,att48,48,90,GurobiTspSolver,0.010,0.064021,33522.0,33431.0,0.002722
302,att48,48,90,GurobiTspSolver,0.050,0.049895,33522.0,33431.0,0.002722
303,att48,48,90,GurobiTspSolver,0.100,0.034748,36559.0,33059.0,0.105871
304,att48,48,90,GurobiTspSolver,0.250,0.022031,36559.0,31669.0,0.154410
...,...,...,...,...,...,...,...,...,...
155,pr439,439,90,CpSatTspSolverMtz,0.001,185.639632,834126.0,93868.0,7.886159
156,pr439,439,90,CpSatTspSolverMtz,0.010,183.571639,742170.0,93833.0,6.909477
157,pr439,439,90,CpSatTspSolverMtz,0.050,186.742688,866567.0,93833.0,8.235205
158,pr439,439,90,CpSatTspSolverMtz,0.100,183.098925,780813.0,93749.0,7.328761


In [4]:
for entry in Benchmark(EXPERIMENT_DATA):
    if (
        entry["parameters"]["args"]["instance_name"] == "random_euclidean_100_1"
        and entry["parameters"]["args"]["strategy"] == "GurobiTspSolver"
    ):
        print("=====================================")
        stdout = "".join(e[1] for e in entry["stdout"])
        stderr = "".join(e[1] for e in entry["stderr"])
        print(stdout)
        print(stderr)
        if not stdout.strip():
            print("No stdout")
        print("=====================================")

## Check for errors in the data

You always want to check if the results you got are actually feasible. Errors easily happen and are not always visible on the plots.
Thus, you want to do some basic checks to detect errors early on. For example, you could accidentally have swapped lower and upper bounds in the data generation process.
Depending on your plots, this may not be visible, and you may end up comparing the wrong data and draw the wrong conclusions.
Or, you could have accidentally swapped runtime and objective values, which could look reasonable in the data as the runtime and the objective often increase with the instance size.

A very basic check is to check if the best lower and upper bounds do not contradict each other. Many errors will be caught by this check. However, you often need some tolerance to account for numerical errors.

In [5]:
assert (t.dropna()["opt_gap"] >= -0.0001).all(), "Optimality gap is negative!"

In [6]:
# Always make sure that your results are not trivially wrong
#  - e.g. lower bound is higher than objective
max_lb = t.groupby(["instance_name"])["lower_bound"].max()
min_obj = t.groupby(["instance_name"])["objective"].min()
eps = 0.0001  # some tolerance is needed when working with floats.
bad_instances = max_lb[max_lb - min_obj > eps * max_lb].index.to_list()
from IPython.display import display

display(t[t["instance_name"].isin(bad_instances)])
assert len(bad_instances) == 0, "Bad instances detected: {}".format(bad_instances)

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
